In [ ]:
!pip install joblib pandas numpy scikit-learn librosa

import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import librosa
import os

In [ ]:
# Định nghĩa hàm ánh xạ valence và arousal sang nhãn cảm xúc
def map_to_emotion(valence, arousal):
    if valence > 5 and arousal > 5:
        return 'HAPPY'
    elif valence < 5 and arousal < 5:
        return 'SAD'
    elif valence > 5 and arousal > 7:
        return 'ENERGY'
    elif valence > 5 and 3 < arousal < 7:
        return 'ROMANTIC'
    elif 3 < valence < 7 and arousal < 3:
        return 'CHILL'
    else:
        return 'OTHER'

# Hàm trích xuất đặc trưng từ file MP3
def extract_features_from_mp3(mp3_path):
    try:
        # Tải file audio
        y, sr = librosa.load(mp3_path, sr=None)

        # Trích xuất các đặc trưng
        mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)  # 13 đặc trưng
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)     # 12 đặc trưng
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr), axis=1)  # 7 đặc trưng
        tempo = librosa.beat.tempo(y=y, sr=sr)[0]                         # 1 đặc trưng
        rms = np.mean(librosa.feature.rms(y=y))                            # 1 đặc trưng
        zcr = np.mean(librosa.feature.zero_crossing_rate(y=y))            # 1 đặc trưng

        # Gộp các đặc trưng thành một vector
        features = np.concatenate((mfcc, chroma, spectral_contrast, [tempo, rms, zcr]))
        return features  # Tổng cộng 35 đặc trưng
    except Exception as e:
        print(f"Error processing {mp3_path}: {e}")
        return None

# Chuẩn bị dữ liệu huấn luyện
def prepare_training_data(annotation_path, audio_dir):
    annotations = pd.read_csv(annotation_path)
    annotations.columns = annotations.columns.str.strip()  # Loại bỏ khoảng trắng trong tên cột

    X = []
    y = []

    for index, row in annotations.iterrows():

        if (index + 1) % 10 == 0:
            print(f"Đang xử lý bài hát {index + 1}/{len(annotations)}: {row['song_id']}")

        song_id = row['song_id']
        mp3_path = os.path.join(audio_dir, f"{int(song_id)}.mp3")  # Giả sử file MP3 có tên theo song_id
        if os.path.exists(mp3_path):
            features = extract_features_from_mp3(mp3_path)
            if features is not None:
                X.append(features)
                emotion = map_to_emotion(row['valence_mean'], row['arousal_mean'])
                y.append(emotion)
        else:
            print(f"File not found: {mp3_path}")

    return np.array(X), np.array(y)

# Huấn luyện mô hình
def train_model(X, y):
    # Chia dữ liệu thành tập huấn luyện và kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f"Kích thước tập huấn luyện: {X_train.shape}, Kích thước tập kiểm tra: {X_test.shape}")
    print("Phân bổ các loại cảm xúc trong tập huấn luyện:")
    print(pd.Series(y_train).value_counts())
    print("Phân bổ các loại cảm xúc trong tập kiểm tra:")
    print(pd.Series(y_test).value_counts())

    # Chuẩn hóa đặc trưng
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Định nghĩa các siêu tham số để thử nghiệm với Grid Search
    param_grid = {
        'n_estimators': [50, 100, 200, 400, 500, 700, 1000],           # Số cây trong rừng
        'max_depth': [None, 10, 20, 50, 70, 90, 100, 200],              # Độ sâu tối đa của cây
        'min_samples_split': [2, 5, 10, 40, 60, 70, 100],          # Số mẫu tối thiểu để chia nút
        'min_samples_leaf': [1, 2, 4, 8, 10, 20, 40, 50, 70],            # Số mẫu tối thiểu ở lá
        'max_features': ['auto', 'sqrt', 'log2']  # Số đặc trưng tối đa được xem xét
    }

    # Khởi tạo mô hình Random Forest
    rf = RandomForestClassifier(random_state=42)

    # Sử dụng GridSearchCV để tìm tham số tối ưu
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=4, verbose=2, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # In ra tham số tối ưu
    print("Tham số tối ưu nhất:", grid_search.best_params_)
    print("Độ chính xác tốt nhất (cross-validation):", grid_search.best_score_)

    # Lấy mô hình tốt nhất từ Grid Search
    best_model = grid_search.best_estimator_

    # Đánh giá mô hình trên tập kiểm tra
    y_pred = best_model.predict(X_test)
    print("Đánh giá mô hình trên tập kiểm tra:")
    print(classification_report(y_test, y_pred))

    # Lưu đặc trưng
    # np.save('/kaggle/input/music-detection-dataset/random_forest_X_train.npy', X_train)
    # np.save('/kaggle/input/music-detection-dataset/random_forest_y_train.npy', y_train)

    return best_model, scaler

# Hàm dự đoán cảm xúc cho bài hát mới
def predict_emotion(mp3_path, model, scaler):
    features = extract_features_from_mp3(mp3_path)
    if features is not None:
        features = scaler.transform([features])  # Chuẩn hóa đặc trưng
        emotion = model.predict(features)[0]
        return emotion
    else:
        return "Error"


In [ ]:
# Đường dẫn đến file annotation và thư mục chứa file MP3
annotation_path = '/kaggle/input/music-detection-dataset/DEAM_Annotations/annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_1_2000.csv'  # Thay bằng đường dẫn thực tế
audio_dir = '/kaggle/input/music-detection-dataset/DEAM_audio/MEMD_audio'  # Thay bằng đường dẫn thực tế đến thư mục chứa file MP3

# Đường dẫn đến file đặc trưng đã lưu
X_train_path = '/kaggle/input/music-detection-dataset/random_forest_X_train.npy'
y_train_path = '/kaggle/input/music-detection-dataset/random_forest_y_train.npy'

# Bước 1: Chuẩn bị dữ liệu và huấn luyện mô hình
print("Đang chuẩn bị dữ liệu huấn luyện...")

# Nếu chưa có đặc trưng, bỏ comment này để trích xuất đặc trưng
#X, y = prepare_training_data(annotation_path, audio_dir)

# Bước 1: Tải dữ liệu đặc trưng đã lưu
print("Đang tải dữ liệu đặc trưng đã lưu...")
X = np.load(X_train_path)
y = np.load(y_train_path)

if len(X) == 0 or len(y) == 0:
    print("Không tải được dữ liệu. Vui lòng kiểm tra thư mục và file dữ liệu.")
else:
    print("Đang huấn luyện mô hình...")
    model, scaler = train_model(X, y)

    joblib.dump(model, 'random_forest_emotion_model.pkl')
    joblib.dump(scaler, 'random_forest_scaler.pkl')


In [ ]:
# Tải lại mô hình và scaler để sử dụng
model_loaded = joblib.load('random_forest_emotion_model.pkl')
scaler_loaded = joblib.load('random_forest_scaler.pkl')

mp3_folder_path = "/kaggle/input/music-detection-dataset/TestSong/TestSong"
output_file_path = 'music_emotion_analysis.txt'


if os.path.exists(mp3_folder_path) and os.path.isdir(mp3_folder_path):
    results = []  # Danh sách để lưu trữ kết quả
    print(f"Đang duyệt các file nhạc trong thư mục: {mp3_folder_path}")
    for filename in os.listdir(mp3_folder_path):
        if filename.endswith(('.mp3', '.wav', '.flac')):  # Lọc các file nhạc phổ biến
            file_path = os.path.join(mp3_folder_path, filename)
            print(f"Đang phân tích file: {filename}...")
            predicted_emotion = predict_emotion(file_path, model_loaded, scaler_loaded)
            print(f"Cảm xúc dự đoán của {filename} là: {predicted_emotion}")
            results.append(f"File: {filename}, Cảm xúc dự đoán: {predicted_emotion}")

    print("Hoàn tất phân tích tất cả các file nhạc.")

    # Lưu kết quả ra file txt trong thư mục /kaggle/working/
    with open(output_file_path, 'w', encoding='utf-8') as f:
        f.write("--- Kết quả phân tích cảm xúc âm nhạc ---\n")
        for result in results:
            f.write(result + '\n')
        f.write("--- Kết thúc ---\n")

    print(f"Kết quả phân tích đã được lưu vào file: {output_file_path}")


else:
    print("Đường dẫn thư mục không hợp lệ hoặc không tồn tại. Vui lòng kiểm tra lại đường dẫn!")

